In [1]:
import psycopg2

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic4'
hostname = 'localhost'
port_number = 5434
schema_name = 'omop_cdm'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, port=port_number, password='mysecretpassword')

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

In [10]:
import pandas as pd

countsQuery = """
    select
    count(*) as total_count,
    count(distinct domain_id) as total_domains,
    count(distinct vocabulary_id) as total_vocabularies,
    count(distinct concept_class_id) as total_concept_classes,
    count(distinct concept_name) as total_concepts
    from
    voc_dataset.concept
    ;
"""
countsDf = pd.read_sql_query(countsQuery, con)
countsDf

,total_count,total_domains,total_vocabularies,total_concept_classes,total_concepts
0,5591082,33,82,184,4104016


In [18]:
import pandas as pd

countsQuery = """
    select
    domain_id as "Domain",
    count(*) as "Domain Count",
    sum(case when standard_concept = 'S' then 1 else 0 end) as "Standard Count",
    sum(case when standard_concept = 'S' then 0 else 1 end) as "Customised Count"
    from
    voc_dataset.concept
    where invalid_reason is null
    group by domain_id
    order by "Domain Count" desc
    ;
"""
countsDf = pd.read_sql_query(countsQuery, con)
countsDf

/home/yram0006/phd/chapter_1/workspace/EHR-standards/.venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Domain,Domain Count,Standard Count,Customised Count
0,Drug,3449769,1920734,1529035
1,Device,219481,218153,1328
2,Condition,218354,112889,105465
3,Observation,209491,137986,71505
4,Geography,203352,203352,0
5,Measurement,138256,91891,46365
6,Procedure,56223,53242,2981
7,Spec Anatomic Site,34702,34695,7
8,Meas Value,19110,19080,30
9,Metadata,3357,992,2365


In [19]:
import pandas as pd

ethnicityQuery = """
    select * from voc_dataset.concept where domain_id = 'Ethnicity';
"""
ethnicityDf = pd.read_sql_query(ethnicityQuery, con)
ethnicityDf

/home/yram0006/phd/chapter_1/workspace/EHR-standards/.venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,load_table_id,load_row_id
0,2000001408,HISPANIC/LATINO,Ethnicity,mimiciv_per_ethnicity,Ethnicity,None,HISPANIC/LATINO,1970-01-01,2099-12-31,None,tmp_custom_mapping,NaN
1,38003563,Hispanic or Latino,Ethnicity,Ethnicity,Ethnicity,S,Hispanic,1970-01-01,2099-12-31,None,concept,1.160080e+09
2,38003564,Not Hispanic or Latino,Ethnicity,Ethnicity,Ethnicity,S,Not Hispanic,1970-01-01,2099-12-31,None,concept,5.698891e+08


In [20]:
import pandas as pd

ethnicityQuery = """
    select
    distinct PER.ethnicity_concept_id
    from
    omop_cdm.person PER
    ;
"""
ethnicityDf = pd.read_sql_query(ethnicityQuery, con)
ethnicityDf

/home/yram0006/phd/chapter_1/workspace/EHR-standards/.venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ethnicity_concept_id
0,0
1,38003563


In [21]:
import pandas as pd

raceQuery = """
    select * from voc_dataset.concept where domain_id = 'Race';
"""
raceDf = pd.read_sql_query(raceQuery, con)
raceDf

/home/yram0006/phd/chapter_1/workspace/EHR-standards/.venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,load_table_id,load_row_id
0,38003607,Tobagoan,Race,Race,Race,S,3.10,1970-01-01,2099-12-31,None,concept,1.749408e+09
1,38003608,Trinidadian,Race,Race,Race,S,3.11,1970-01-01,2099-12-31,None,concept,-2.793061e+07
2,8657,American Indian or Alaska Native,Race,Race,Race,S,1,1970-01-01,2099-12-31,None,concept,1.783987e+09
3,38003572,American Indian,Race,Race,Race,S,1.01,1970-01-01,2099-12-31,None,concept,1.016136e+09
4,38003573,Alaska Native,Race,Race,Race,S,1.02,1970-01-01,2099-12-31,None,concept,5.091420e+08
...,...,...,...,...,...,...,...,...,...,...,...,...
766,2000001402,OTHER,Race,mimiciv_per_ethnicity,Race,S,OTHER,1970-01-01,2099-12-31,None,tmp_custom_mapping,NaN
767,2000001403,ASIAN,Race,mimiciv_per_ethnicity,Race,None,ASIAN,1970-01-01,2099-12-31,None,tmp_custom_mapping,NaN
768,2000001407,AMERICAN INDIAN/ALASKA NATIVE,Race,mimiciv_per_ethnicity,Race,None,AMERICAN INDIAN/ALASKA NATIVE,1970-01-01,2099-12-31,None,tmp_custom_mapping,NaN
769,2000001404,WHITE,Race,mimiciv_per_ethnicity,Race,None,WHITE,1970-01-01,2099-12-31,None,tmp_custom_mapping,NaN


In [27]:
import pandas as pd

raceQuery = """
    select
    PER.race_concept_id as race_concept_id,
    CON.concept_code as concept_code,
    CON.concept_name as concept_name,
    CON.vocabulary_id as vocabulary_id,
    count(*) as total_count
    from
    omop_cdm.person PER
    left join voc_dataset.concept CON
    on PER.race_concept_id = CON.concept_id
    group by PER.race_concept_id, CON.vocabulary_id, CON.concept_code, CON.concept_name
    ;
"""
raceDf = pd.read_sql_query(raceQuery, con)
raceDf

/home/yram0006/phd/chapter_1/workspace/EHR-standards/.venv/lib/python3.9/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,race_concept_id,concept_code,concept_name,vocabulary_id,total_count
0,0,No matching concept,No matching concept,None,94444
1,8515,2,Asian,Race,16446
2,8516,3,Black or African American,Race,30946
3,8527,5,White,Race,158683
4,8657,1,American Indian or Alaska Native,Race,940
5,2000001401,UNKNOWN,UNKNOWN,mimiciv_per_ethnicity,17069
6,2000001402,OTHER,OTHER,mimiciv_per_ethnicity,16040
7,2000001405,UNABLE TO OBTAIN,UNABLE TO OBTAIN,mimiciv_per_ethnicity,3374
